In [1]:
import math
import numpy as np
import time

import cv2 as cv
import mediapipe as mp

In [2]:
def getAngle(p1, p2, p3, frame, draw):
    fh,fw = frame.shape[:2]
    x1,y1 = int(p1.x*fw), int(p1.y*fh)
    x2,y2 = int(p2.x*fw), int(p2.y*fh)
    x3,y3 = int(p3.x*fw), int(p3.y*fh)
    
    angle = abs(int(math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2, x1-x2))))
    if angle>180:
        angle = 180
    
    if draw:
        cv.line(frame, (x1,y1), (x2,y2), (0,0,255), 2)
        cv.line(frame, (x2,y2), (x3,y3), (0,0,255), 2)
        cv.circle(frame, (x1,y1), 5, (255,255,255), -1)
        cv.circle(frame, (x1,y1), 10, (255,255,255), 2)
        cv.circle(frame, (x2,y2), 5, (255,255,255), -1)
        cv.circle(frame, (x2,y2), 10, (255,255,255), 2)
        cv.circle(frame, (x3,y3), 5, (255,255,255), -1)
        cv.circle(frame, (x3,y3), 10, (255,255,255), 2)
        cv.putText(frame, str(angle), (x2-30,y2+30), cv.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 2)
    
    return angle

In [3]:
def Curltracking(path=0, righthand=True, curlstart=150, curlend=38, showangle=True, showfps=True, drawLand=False, min_detection_confidence=0.75, min_tracking_confidence=0.5, drawland=False, wCam=640, hCam=488):

    video = cv.VideoCapture(path)
    video.set(3, wCam)
    video.set(4, hCam)

    mppose = mp.solutions.pose
    poses = mppose.Pose(min_detection_confidence=min_detection_confidence, min_tracking_confidence=min_tracking_confidence)
    mpdraw = mp.solutions.drawing_utils

    poselms = []

    curl = False
    curlcounter = 0

    pTime = 0
    cTime = 0

    while True:
        success, frame = video.read()

        if success:

            rgbframe = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            result = poses.process(rgbframe)

            if result.pose_landmarks:
                if drawLand:
                    mpdraw.draw_landmarks(frame, result.pose_landmarks, mppose.POSE_CONNECTIONS)
                poselms = result.pose_landmarks.landmark
                if righthand:
                    angle = getAngle(poselms[12], poselms[14], poselms[16], frame, showangle)
                else:
                    angle = getAngle(poselms[11], poselms[13], poselms[15], frame, showangle)
                per = np.interp(angle, (curlend,curlstart), (100,0))
                # Curl only gets registered when it starts from curlstart angle and ends at curlend angle
                if per==0:
                    curl = True
                if(curl and per==100):
                    curlcounter+=1
                    curl = False
                cv.putText(frame, 'Counter: '+str(curlcounter), (10,80), cv.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                per = int((frame.shape[0]-10-120)*(1-per/100))
                cv.rectangle(frame, (10,frame.shape[0]-10), (20,120), (0,0,255), 2)
                cv.rectangle(frame, (10,frame.shape[0]-10), (20,120+per), (0,0,255), -1)

            else:
                poselms = []
                # If tracking breaks curl resets
                curl = False
                curlcounter = 0

            if showfps:
                cTime = time.time()
                fps = 1/(cTime-pTime)
                pTime = cTime

                cv.putText(frame, 'FPS: '+str(int(fps)), (10,40), cv.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

            cv.imshow('Video', frame)

            if(cv.waitKey(1) & 0xFF==ord('d')):
                break

        else:
            break
    video.release()
    cv.destroyAllWindows()

## Curltracking Method

* **path -** path of the video source. 0: webcam / 1,2,..: connected media number / '../path': video file path. (default: 0)
* **righthand -** to track right/left hand with values True/False correspondingly. (default: True)
* **curlstart -** curl starting angle in degrees. (default: 150)
* **curlend -** curl ending angle in degrees. (default: 25)
* **showangle -** show tracking of curl arm with angles. (default: True)
* **show_fps -** show fps of running video. (default: False)
* **drawLand -** draw pose landmarks. (default: False)
* **min_detection_confidence -** minimum detection confidence. (default: 0.75)
* **min_tracking_confidence -** minimum tracking confidence. (default: 0.5)
* **wCam -** set width of captured video source. (default: 640)
* **hCam -** set height of captured video source. (default: 480)

In [5]:

Curltracking('curl.mp4', showangle=False)